In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import csv
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

In [14]:
img_width, img_height = 150, 150

train_data_dir = 'CelebA_Spoof/Data/train'
validation_data_dir = 'CelebA_Spoof/Data/test'
nb_train_samples = 1052
nb_validation_samples = 200
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)


model.save_weights('first_try.h5')

Found 1052 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/50
65/65 [==============================] - 80s 1s/step - loss: 0.5568 - accuracy: 0.7384 - val_loss: 0.5247 - val_accuracy: 0.7656
Epoch 2/50
65/65 [==============================] - 78s 1s/step - loss: 0.3388 - accuracy: 0.8793 - val_loss: 0.4429 - val_accuracy: 0.7292
Epoch 3/50
65/65 [==============================] - 84s 1s/step - loss: 0.2911 - accuracy: 0.8948 - val_loss: 0.4017 - val_accuracy: 0.7812
Epoch 4/50
65/65 [==============================] - 86s 1s/step - loss: 0.2602 - accuracy: 0.8967 - val_loss: 0.3263 - val_accuracy: 0.8490
Epoch 5/50
65/65 [==============================] - 99s 2s/step - loss: 0.2173 - accuracy: 0.9141 - val_loss: 0.3366 - val_accuracy: 0.8177
Epoch 6/50
65/65 [==============================] - 86s 1s/step - loss: 0.2029 - accuracy: 0.9170 - val_loss: 0.2738 - val_accuracy: 0.9062
Epoch 7/50
65/65 [==============================] - 76s 1s/step - loss: 0.191

In [16]:
model.evaluate_generator(
  validation_generator,
  nb_validation_samples/batch_size,
  workers = 1
)

[0.20748330652713776, 0.9200000166893005]